In [1]:
import os
from scripts.Unitex import Unitex
from scripts.posTagger_to_unitex import spacy2unitex
import spacy
from bs4 import BeautifulSoup

In [ ]:
# pip install -U spacy
#!python -m spacy download en_core_web_sm
#!python -m spacy download fr_core_news_sm

## Create Unitex input txt file from spaCy doc

In [26]:
# Load French tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

# Process whole documents
text = ("13th April 201434")
doc = nlp(text)

unitex_input = spacy2unitex(doc)
print(unitex_input)
filename = 'tmp'
filepath = os.path.join('output', filename + '.txt')    
with open(filepath, 'w') as f:
    f.write(unitex_input)


{13th,13th.NOUN} {April,April.PROPN} {201434,201434.NUM} 


## Run Unitex as a Python code snippet

In [27]:
version = "Tutorial"
lang = 'English'
install_path = "/Users/lmoncla/Nextcloud-LIRIS/Programmes/Unitex-GramLab-3.2"
install_path_app = "/Users/lmoncla/Programmes/Unitex-GramLab-3.2/App"
filepath = os.path.join('output', filename)

delete_tmp_files = True

unitex = Unitex(version, lang, install_path, install_path_app, delete_tmp_files)
doc = unitex.run(filepath)


In [19]:
doc

'<?xml version=\'1.0\' encoding=\'utf-8\'?><doc><w pos="NUM" lemma="13">13</w> <date><w pos="PROPN" lemma="April">April</w></date> <w pos="NUM" lemma="2014">2014</w> </doc>'

## Run `Unitex.py` as a Python script

In [28]:
version = "Tutorial"
lang = 'English'
install_path = "/Users/lmoncla/Nextcloud-LIRIS/Programmes/Unitex-GramLab-3.2"
install_path_app = "/Users/lmoncla/Programmes/Unitex-GramLab-3.2/App"
filepath = os.path.join('output', filename)

!python scripts/Unitex.py -i $filepath -l $lang -c $version --install_path $install_path --install_path_app $install_path_app

with open(filepath + "_csc_csc.xml", "r") as file:
    doc = file.read()


## Parse the XML result

In [29]:
root = BeautifulSoup(doc, 'xml')
print(root.prettify())

<?xml version="1.0" encoding="utf-8"?>
<doc>
 <w lemma="13th" pos="NOUN">
  13th
 </w>
 <date>
  <w lemma="April" pos="PROPN">
   April
  </w>
  <w lemma="201434" pos="NUM">
   201434
  </w>
 </date>
</doc>


In [6]:
print('text:', text)

for element in root.find_all('date'):
    print('date:', end=' ')
    for w in element.find_all('w'):
        print(w.string, end=' ')
print()

text: 13th April 2014
date: April 2014 
